<h1><center> 2.3a Reduce solid waste per person</center></h1>

<center>Goal: sum recycle to county level</center>

<h3>Process the population data</h3>

In [2]:
import pandas as pd

In [3]:
#pop_morpc.columns.astype(object)
#Store in xlsx
#pop.to_excel('County_pop_2000_2018.xlsx')
#pop_morpc.to_excel('Morpc_County_pop_2000_2018.xlsx')
pop = pd.read_excel('MORPC_Region_County_Population_2000_2019_wide.xlsx')

In [74]:
print(pop.columns)
pop.set_index(['COUNTY'],inplace = True)
pop.head()


Index(['COUNTY',    'Id2',     2000,     2001,     2002,     2003,     2004,
           2005,     2006,     2007,     2008,     2009,     2010,     2011,
           2012,     2013,     2014,     2015,     2016,     2017,     2018,
           2019],
      dtype='object')


,Id2,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
COUNTY,,,,,,,,,,,,,,,,,,,,,
DELAWARE,39041.0,111714,119098,127011,134899,141958,149353,155669,160698,164913,...,175098,178517,181124,185383,189451,193298,196923,200542,204826,208974
FAIRFIELD,39045.0,123414,125963,128860,131982,135055,137457,139503,141239,142299,...,146399,147141,147279,148783,150355,151202,152656,154557,155782,157155
FAYETTE,39047.0,28434,28170,28152,28133,28078,28212,28227,28272,28198,...,29017,28865,28765,28683,28647,28598,28658,28655,28666,28612
FRANKLIN,39049.0,1072307,1082421,1087369,1092518,1096256,1101224,1111018,1122357,1135643,...,1166261,1180741,1199071,1218985,1238527,1257552,1274580,1295706,1310300,1325083
HOCKING,39073.0,28237,28364,28511,28608,28687,28718,28699,28989,28968,...,29480,29466,29306,28653,28745,28498,28388,28457,28385,28362


<h3>Process the EPA Solid Waste data</h3>

In [27]:
df2006 = pd.read_excel('EPA Solid Waste 2006_2018.xlsx',str(2006))
df2006['year'] = 2006
df2006.head()



,SENDCNTY,COUNTY,Facility,TW_DH,TW_CoreID,GENERAL,ASBESTOS,RESIDUAL,INDUSTRIAL,CD_D,EXEMPT,RECYCLABLES,OTHER,year
0,21,DELAWARE,Richland County Transfer Station,NaN,0,1.86,0.0,0,0.0,0.00,0.0,0.0,0.0,2006
1,21,DELAWARE,Reynolds Avenue Transfer Station,NaN,0,31335.82,0.0,0,0.0,2413.33,0.0,0.0,0.0,2006
2,21,DELAWARE,Johnson Transfer & Recycling,NaN,0,7534.66,0.0,0,0.0,280.45,0.0,383.0,0.0,2006
3,21,DELAWARE,Rumpke Transfer Facility,NaN,0,22124.01,0.0,0,0.0,10.52,0.0,0.0,0.0,2006
4,21,DELAWARE,Ross Brothers Waste & Recycling Co.,NaN,0,1664.72,0.0,0,0.0,0.00,0.0,0.0,0.0,2006


In [92]:

recycledf = df2006.loc[:,['COUNTY','RECYCLABLES','year']]
print(len(recycledf))
print(recycledf.isnull().sum())
recycledf.dropna(subset=['COUNTY'],inplace = True)
recycledf['RECYCLABLES'] = recycledf['RECYCLABLES'].fillna(0)
print(len(recycledf))
recycle = recycledf[recycledf['COUNTY'].isin(['MARION','LOGAN', 'UNION', 'MADISON', 'FAYETTE', 'DELAWARE', 'MORROW', 'FRANKLIN','PICKAWAY', 'ROSS', 'KNOX', 'LICKING', 'FAIRFIELD', 'PERRY', 'HOCKING'])]
print(len(recycle))
df_sum = recycle.groupby(['COUNTY','year'])['RECYCLABLES'].sum().reset_index()

213
COUNTY         0
RECYCLABLES    0
year           0
dtype: int64
213
208


In [31]:
df_sum['RECYCLABLES'] = df_sum.RECYCLABLES.astype(float)
#recycle from tons to bound
df_sum['RECYCLABLES_lbs'] = df_sum['RECYCLABLES']*2000
df_sum.head()

,COUNTY,year,RECYCLABLES,RECYCLABLES_lbs
0,DELAWARE,2006,383.00,766000.0
1,FAIRFIELD,2006,2321.24,4642480.0
2,FAYETTE,2006,18.51,37020.0
3,FRANKLIN,2006,36218.44,72436880.0
4,HOCKING,2006,2.95,5900.0


In [33]:
# region
region = df_sum.groupby(['year'])['RECYCLABLES_lbs'].sum().reset_index()
region

,year,RECYCLABLES_lbs
0,2006,86979020.0


In [78]:
region['pop'] = pop.loc['15-County Region',2006]
region['RECYCLABLES_per_cap'] = region['RECYCLABLES_lbs']/region['pop']
final_region = region.loc[:,['year','RECYCLABLES_per_cap']]
final_region

,year,RECYCLABLES_per_cap
0,2006,41.930856


In [82]:
#county
pop_county = pop.loc[:,['Id2',2006]]
pop_county.head()

,Id2,2006
COUNTY,,
DELAWARE,39041.0,155669
FAIRFIELD,39045.0,139503
FAYETTE,39047.0,28227
FRANKLIN,39049.0,1111018
HOCKING,39073.0,28699


In [85]:
county_merge = pd.merge(df_sum,pop_county,on='COUNTY')
county_merge['recycle_per_cap'] = county_merge['RECYCLABLES_lbs']/county_merge[2006]

In [90]:
county_merge.head()

,COUNTY,year,RECYCLABLES,RECYCLABLES_lbs,Id2,2006,recycle_per_cap
0,DELAWARE,2006,383.00,766000.0,39041.0,155669,4.920697
1,FAIRFIELD,2006,2321.24,4642480.0,39045.0,139503,33.278711
2,FAYETTE,2006,18.51,37020.0,39047.0,28227,1.311510
3,FRANKLIN,2006,36218.44,72436880.0,39049.0,1111018,65.198656
4,HOCKING,2006,2.95,5900.0,39073.0,28699,0.205582


In [93]:
final_county = county_merge.loc[:,['Id2','year','recycle_per_cap']]
final_county['Id2'] =  final_county.Id2.astype(object)
final_county.head()

,Id2,year,recycle_per_cap
0,39041,2006,4.920697
1,39045,2006,33.278711
2,39047,2006,1.311510
3,39049,2006,65.198656
4,39073,2006,0.205582


In [13]:
xls = pd.ExcelFile('EPA Solid Waste 2006_2018.xlsx')
pop = pd.read_excel('MORPC_Region_County_Population_2000_2019_wide.xlsx')
pop.set_index(['COUNTY'],inplace = True)
pop.rename(columns=lambda x: str(x), inplace=True)
#2006 to 2018
def readsheet(sheetname,pop):
    #process population
    df = pd.read_excel(xls, sheetname)
    df['year'] = sheetname
    df= df.loc[:,['COUNTY','RECYCLABLES','year']]

    df.dropna(subset=['COUNTY'],inplace = True)
    df['RECYCLABLES'] = df['RECYCLABLES'].fillna(0)

    recycle = df[df['COUNTY'].isin(['MARION','LOGAN', 'UNION', 'MADISON', 'FAYETTE', 'DELAWARE', 'MORROW', 'FRANKLIN','PICKAWAY', 'ROSS', 'KNOX', 'LICKING', 'FAIRFIELD', 'PERRY', 'HOCKING'])]
    df_sum = recycle.groupby(['COUNTY','year'])['RECYCLABLES'].sum().reset_index()
    df_sum['RECYCLABLES'] = df_sum.RECYCLABLES.astype(float)
    #recycle from tons to bound
    df_sum['RECYCLABLES_lbs'] = df_sum['RECYCLABLES']*2000
    #region
    region = df_sum.groupby(['year'])['RECYCLABLES_lbs'].sum().reset_index()
    region['pop'] = pop.loc['15-County Region',sheetname]
    region['RECYCLABLES_per_cap'] = region['RECYCLABLES_lbs']/region['pop']
    final_region = region.loc[:,['year','RECYCLABLES_per_cap']]
    #county
    pop_county = pop.loc[:,['Id2',sheetname]]
    county_merge = pd.merge(df_sum,pop_county,on='COUNTY')
    county_merge['recycle_per_cap'] = county_merge['RECYCLABLES_lbs']/county_merge[sheetname]
    final_county = county_merge.loc[:,['Id2','year','recycle_per_cap']]
    final_county['Id2'] =  final_county.Id2.astype(object)
    
    return final_region, final_county

In [14]:
region,county = readsheet('2006',pop)

In [15]:
region

,year,RECYCLABLES_per_cap
0,2006,41.930856


In [16]:
county.head()

,Id2,year,recycle_per_cap
0,39041,2006,4.920697
1,39045,2006,33.278711
2,39047,2006,1.311510
3,39049,2006,65.198656
4,39073,2006,0.205582


In [17]:
year = [ str(i) for i in range(2006,2019)]
df_region = pd.DataFrame(columns=['year','RECYCLABLES_per_cap'])
df_county = pd.DataFrame()
for i in year:
    region,df = readsheet(i,pop)
    #df_county = pd.concat([df_county,df],ignore_index=True)
    df_county = df_county.append(df)
    df_region = df_region.append(region)

In [26]:
df_region

,year,RECYCLABLES_per_cap
0,2006,41.930856
0,2007,10.688202
0,2008,26.458774
0,2009,7.364200
0,2010,7.855949
0,2011,10.558261
0,2012,9.498743
0,2013,9.386615
0,2014,7.570446
0,2015,11.387870


In [27]:
print(len(df_county))
df_county

194


,Id2,year,recycle_per_cap
0,39041,2006,4.920697
1,39045,2006,33.278711
2,39047,2006,1.311510
3,39049,2006,65.198656
4,39073,2006,0.205582
...,...,...,...
10,39117,2018,0.000000
11,39127,2018,0.000000
12,39129,2018,0.065076
13,39141,2018,0.000000
